![Регрессия](https://zr.ru/_ah/img/vgtv7RJ4pcu8y7Lq847wYw=s800 "Предсказываем цены")

## Прогнозируем стоимость автомобиля по его характеристикам

> По условиям соревнования обучающую выборку мы должны собрать сами. В данной работе использованы данные от **8.11.2020** с сайта auto.ru.

Целевой метрикой выбрана MAPE:

$$
MAPE = \frac{100}{n}\sum_i\left \| \frac{y(i)-\hat{y}(i)}{y} \right \|
$$

# Подкючаем библиотеки

In [ ]:
import os
import shap # SHAPley
import secrets # rnd str generator
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import make_scorer
from lightgbm import LGBMRegressor
import category_encoders as ce
import xgboost as xgb

import eli5
from eli5.sklearn import PermutationImportance
import lime
import lime.lime_tabular

from sklearn.decomposition import PCA
from collections import Counter
import joblib
import ast

from catboost import Pool, CatBoostRegressor

from pandas_profiling import ProfileReport

# Загрузим собственные модули
from shutil import copyfile
copyfile(src = "../input/myutils/dshelper.py", dst = "../working/dshelper.py")
from dshelper import *

## SKLEARN
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder

# Константы 

In [ ]:
VERSION    = 3
DIR_TRAIN  = '../input/train-data/'
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# CATBOOST
ITERATIONS = 5000
LR         = 0.1

RANDOM_SEED = 42
N_FOLDS = 5
CURRENT_YEAR = 2020

FIGSIZE = (5, 2)

In [ ]:
# Отчеты по каждой CV модели
def show_models_report():
    
    frames = []
    
    for file in os.listdir():
        if file.endswith('.frame'):
            filename = file[:-6]
            report = joblib.load(filename+'.frame')
            num_cols = report.columns[report.columns.str.contains('FOLD_')]
            report[num_cols] = report[num_cols] .astype('float64')
            frames.append(report)
        
    return pd.DataFrame(pd.concat(frames))
    
# ROUND PREDS
def round_preds(y_pred, step=5000):
    return np.array([step*round(x/step) for x in y_pred])

# MAPE
def mean_absolute_percentage_error(y_true, y_pred, **kwargs): 
    y_true, y_pred = np.array(y_true), round_preds(np.array(y_pred))
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# MAPE для отдельного наблюдения
def mape_per_row(y_true_col, y_pred_col):
    return np.abs((y_true_col - y_pred_col) / y_true_col) * 100

# Запуск модели через cross-val
def run_model_cv(model_func=None, X=None, y=None, sub_test=None, cv=5, random_state=42, name='basemodel', comment='', model_type='tree', target_encoding=False, te_cols=None):
    # Попробуем нашу первую базовую модель на CatBoost через CV
    sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')
    submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
    
    # OOF PREDS
    oof_preds = np.zeros(X.shape[0])
    
    # Сохраняем рейтинг модели по каждому фолду
    mape_cols = [f'FOLD_{col+1}' for col in range(cv)]
    name_postfix = secrets.token_urlsafe(4)
    mape = pd.DataFrame([[0 for _ in range(cv)]], columns=mape_cols, index=[name+name_postfix]) # пишем рейтинг по каждой модели
    
    feature_importance_df = pd.DataFrame()

    score_ls = []
    
    # Пробуем разные сплиты
    #splits = list(KFold(n_splits=cv, shuffle=True, random_state=random_state).split(X, y))
    splits = list(StratifiedKFold(n_splits=cv, shuffle=True, random_state=random_state).split(X, y))


    for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=cv,):
        # use the indexes to extract the folds in the train and validation data
        X_train, y_train, X_test, y_test = X.iloc[train_idx], y.iloc[train_idx], X.iloc[test_idx], y.iloc[test_idx]   
            
        # model func or estimator for this fold    
        model = model_func(X_train, X_test, y_train, y_test)
        # score model on test
        test_predict = model.predict(X_test)
        test_score = mean_absolute_percentage_error(y_test, test_predict)
        score_ls.append(test_score)
        
        # OOF Predicts
        oof_preds[test_idx] = test_predict
        
        fold_mape = mean_absolute_percentage_error(y_test, test_predict)
        print(f"{idx+1} Fold Test MAPE: {fold_mape:0.3f}")
        # Save mape to df
        mape[f'FOLD_{idx+1}'] = f'{fold_mape:0.3f}'
        
        # submissions
        submissions[f'sub_{idx+1}'] = model.predict(sub_test)
        model_name = f'{name}_fold_{idx+1}.model'
        joblib.dump(model, model_name)
        #model.save_model(f'catboost_fold_{idx+1}.model')
        
        # Feature importances
        if model_type == 'tree':
            feature_importance = model.feature_importances_
        elif model_type == 'lgbm':
            feature_importance = model.feature_importance()
        else:
            feature_importance = 0.1
            
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = X.columns
        fold_importance_df["importance"] = feature_importance
        fold_importance_df["fold"] = idx + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    print(f'Mean Score: {np.mean(score_ls):0.3f}')
    print(f'Std Score: {np.std(score_ls):0.4f}')
    print(f'Max Score: {np.max(score_ls):0.3f}')
    print(f'Min Score: {np.min(score_ls):0.3f}')
    
    mape['comment'] = comment
    mape['mean'] = np.round(np.mean(score_ls), 3)
    display(mape)

    # Сохраним рейтинг модели
    joblib.dump(mape, name+'_'+name_postfix+'_rating.frame')

    return submissions, feature_importance_df, oof_preds


# Объединяем трейн и тест
def merge_dataset(df_train, df_test):
    df_train['sample'] = 1 # помечаем где у нас трейн
    df_test['sample'] = 0 # помечаем где у нас тест
    data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

    return data

# Загрузка данных
def load_data(load_nlp=True):
    '''
        Loads our dataset
        
        - load_nlp - load NLP preprocessed description feature 
    '''
    data = None
    # Наши данные
    df_test = pd.read_csv(DIR_TEST + 'test.csv')
    df_train = pd.read_csv(DIR_TRAIN + 'train_data.csv', sep='|')

    # Сразу удалим лишние колонки, которые есть в наших данных
    df_train.drop(['currency', 'Комплектация'], axis=1, inplace=True)
    
    data = merge_dataset(df_train, df_test)
    
    data.super_gen = data.super_gen.fillna('{}')
    data.super_gen = data.super_gen.map(ast.literal_eval)

    
    # Загружаем описание объявлений из файла
    if load_nlp:
        description = pd.read_csv(DIR_TRAIN+'description.csv', header=None, sep='\n')
        data.description = description[0].apply(lambda s: s.split(','))
        data.description = data.description.fillna('nodata')
    
    return data
    

# Разделяем датасет на тест и трейн
def data_split(df):
    X = df.query('sample == 1').drop(['sample'], axis=1)
    X_sub = df.query('sample == 0').drop(['sample', 'price'], axis=1)
    
    return X, X_sub


# Обработка датасета через пайплайн
def prepare_data_pipeline(data, processing_pipe):
    '''
        Return Processed DataFrame through pipeline

        return: X, y, X_sub
    '''
    # Обработаем данные
    processing_pipe = processing_pipe

    # Проводим обработку отдельно для теста и трэйна, чтобы избежать даталиков
    data = processing_pipe.fit_transform(data)
    
    # Разбиваем данные на трейн и тест
    X, X_sub = data_split(data)

    y = X.price
    X = X.drop(['price'], axis=1)

    return X, y, X_sub


# Получаем самые частые слова из токенов
def get_most_common_words(text, limit=10):   
    most_common_list = []
    frequencies = Counter(word for sentence in text for word in sentence)
    for word, frequency in frequencies.most_common(limit):  # get the 10 most frequent words
        most_common_list.append(word)
        
    return most_common_list


def plot_feature_by_avg_target(df, feature, target='price', title='Средняя цена (Руб.)', plot=True):
    df_avg_grp = df[[feature, target]].groupby(feature, as_index = False).mean().rename(columns={target:feature+'_avg_'+target})
    
    if plot:
        plt1 = df_avg_grp.plot(x = feature, kind='bar', legend = False, sort_columns = True, figsize = (15,3))
        plt1.set_xlabel(feature)
        plt1.set_ylabel(title)
        plt.xticks(rotation = 90)
        plt.show()
    
    return df_avg_grp

# Данные

In [ ]:
data = load_data()
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

df_train, df_test = data_split(data)

In [ ]:
pd.set_option('display.max_column', 0)
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum() 

Удалим из трейна пустые колонки - currency, Комплектация, image, car_url и две пустые записи.

In [ ]:
# Сразу удалим пару пустых записей
df_train = df_train[~df_train.bodyType.isnull()]

In [ ]:
df_test.isnull().sum()

С остальными пропусками поработаем позднее. Удалим бесполезные для модели признаки из теста: image, car_url

![](https://d1m75rqqgidzqn.cloudfront.net/wp-data/2020/04/09140845/shutterstock_352982963.jpg)
# EDA

## bodyType - тип кузова автомобиля

In [ ]:
compare_features_in_df(df_train, df_test, 'bodyType')

Видно, что данные очень похожи. Видно что к признаку добавлено количество дверей, но у нас есть отдельный признак **numberOfDoors**, поэтому оставим в признаке только тип кузова.              

In [ ]:
df_train['bodyType'] = df_train.bodyType.apply(lambda x: x.split()[0])
df_test['bodyType'] = df_test.bodyType.apply(lambda x: x.split()[0])

In [ ]:
compare_features_in_df(df_train, df_test, 'bodyType')

In [ ]:
# Посмотрим какие значения пересекаются в выборках
get_intersection_df(df_train, df_test, 'bodyType')

In [ ]:
# Посмотрим какие значения не пересекаются
get_intersection_df(df_train, df_test, 'bodyType', reverse=True)

Остался один тип кузова, которого нет в обеих выборках. Можно смело удалять - всего 4 записи.

In [ ]:
data = merge_dataset(df_train, df_test)
df_train.groupby('bodyType').size().plot(kind='pie', textprops={'fontsize': 10})

In [ ]:
plot_boxplots(data, 'bodyType', 'price')

Видно что, присутствуют выбросы почти в каждой категории.

## brand - марка авто

In [ ]:
compare_features_in_df(df_train, df_test, 'brand', show_report=True, limit=1000)

В тестовой выборке присутствует 121 брэнд автомобилей, в то время как в тестовой выборке всего 12 моделей. Кроме того видим, что большой объем данных (более 8000 объявлений) составляет Mercedes-Benz. В тестовой выборке он обозначен как MERCEDES, поэтому необходимо переименовать значение в трейне.

In [ ]:
mercedes = {'MERCEDES-BENZ':'MERCEDES'}
df_train.brand = df_train.brand.replace(mercedes)

In [ ]:
compare_features_in_df(df_train, df_test, 'brand', show_report=False)

С удалением лишних марок поиграемся на этапе валидации моделей.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(20, 5))
plt.xticks(rotation=90)
sns.countplot(data=data.groupby("brand").filter(lambda x: len(x) < 1000), x='brand', ax=axes[0])

# Посмотрим на оставшиеся марки
sns.countplot(data=data.groupby("brand").filter(lambda x: len(x) >= 1000), x='brand')

## color - цвет авто

In [ ]:
compare_features_in_df(df_train, df_test, 'color', show_report=True)

Видим, что в тренировочной выборке значения представлены в HEX, а в тестовой словами. Необходимо обработать данный признак. Для этого составим словарь соответствия HEX-->COLOR.

In [ ]:
colors = {
    'EE1D19': 'красный',
    'FFD600': 'жёлтый',
    'DEA522': 'золотистый',
    'FAFBFB': 'белый',
    'CACECB': 'серебристый',
    '040001': 'чёрный',
    '0000CC': 'синий',
    '97948F': 'серый',
    '200204': 'коричневый',
    '007F00': 'зелёный',
    '660099': 'пурпурный',
    'FF8649': 'оранжевый',
    '22A0F8': 'голубой',
    'C49648': 'бежевый',
    '4A2197': 'фиолетовый',
    'FFC0CB': 'розовый',
    '40001': 'чёрный'
}

df_train.color = df_train.color.replace(colors)

In [ ]:
compare_features_in_df(df_train, df_test, 'color', show_report=True)

Цвета привели в полное соответствие. это наш категориальный признак.

In [ ]:
data = merge_dataset(df_train, df_test)
plot_boxplots(data, 'color', 'price')

## complectation_dict - словарь компектаций

Данный признак содержит JSON с различной информацией о комплектации автомобиля. В данном признаке очень много пустой информации и для дальнейшей работы он нам не пригодится. Просто отмечаем на удаление.

## description - описание объявления

In [ ]:
# Посмотрим одну запись
#df_train.description.value_counts()[:1]

Данный признак содержит описание объявления. Возможно будем использовать как признак для дальнейшей работой с NLP. Оставим пока как есть.

In [ ]:
data = merge_dataset(df_train, df_test)
data.description = data.description.fillna('nodata')
#data.description = data.description.apply(wrap_nlp)

In [ ]:
# Очистим текст от лишних символов и переносов строк
# from bs4 import BeautifulSoup
# import re

# def clear_text(text):
#     text = text.replace('\n', ' ')
#     text = text.replace('\r', ' ')

#     # Отсекаем лишнее
#     result = re.sub('[\W_]+', ' ', text)

#     # Фильтруем совсем маленькое описание - скорее всего мусор
#     if len(result) < 5:
#         result = 'nodata'

#     return BeautifulSoup(result.strip(),"lxml").get_text()

# data.description = data.description.apply(clear_text)

## Лемматизация описания авто
Использование mystem влоб заняло бы на моем компьтере 16 часов. Поэтому пришлось поискать решение в инете. 

Помогла статья https://habr.com/ru/post/503420/

Процесс занимает порядка 10 минут, поэтому сохраняем результат в файл и загружаем в датасет, чтобы не терять время на обработку

In [ ]:
# # Выполним лемматизацию описания
# from pymystem3 import Mystem
# from tqdm import tqdm

# from joblib import Parallel, delayed
# from nltk.corpus import stopwords
# from pymystem3 import Mystem
# from string import punctuation

# batch_size = 1000

# # Объединяем все наши описания в единый список
# texts = data.description.str.cat(sep='|').split('|')[:]

# # Фильтруем стоп-слова
# russian_stopwords = stopwords.words("russian")

# text_batch = [texts[i: i + batch_size] for i in range(0, len(texts), batch_size)]

# def lemmatize(text):

#     m = Mystem()
#     merged_text = "|".join(text)

#     doc = []
#     res = []

#     tokens = m.lemmatize(merged_text)
#     tokens = [token for token in tokens if token not in russian_stopwords \
#                                              and token != " "
#                                              and token != '\n']

#     for t in tokens:
#         if t.strip() != '|':
#             doc.append(t)
#         else:
#             res.append(doc)
#             doc = []
#     else:
#         res.append([t])
    
#     print(f'Input text len: {len(text)}')
#     print(f'Output text len: {len(res)}')

#     return res

# Вот здесь тоже немного магии :)
#processed_texts = Parallel(n_jobs=-1)(delayed(lemmatize)(t) for t in tqdm(text_batch))


In [ ]:
# def list_to_csv(data=None, filename='data'):

#     if data is not None and isinstance(data, list):
#         import csv

#         with open(filename+'.csv', 'w+', encoding='utf8', newline ='') as file:
#             with file:     
#                 write = csv.writer(file) 
#                 write.writerows(data) 

# texts = []

# for batch in tqdm(processed_texts):
#     for text in batch:
#         texts.append(text)


In [ ]:
from collections import Counter

# Загрузим подготовенный csv с описанием
description = pd.read_csv(DIR_TRAIN+'description.csv', header=None, sep='\n')
data.description = description[0].apply(lambda s: s.split(','))
data.description = data.description.fillna('nodata') # 2 записи с NaN

In [ ]:
# # Самые частые слова
# top_desc_words = get_most_common_words(data.description, 1000)

# # Отфильтруем описание по самым частым словам
# data.description = data.description.apply(lambda s: set([x for x in s if x in top_desc_words]))

In [ ]:
# Создадим признаки из description
#desc_bins = get_binary_dummies(data, 'description')

In [ ]:
# Вернем обратно train, test
df_train, df_test = data_split(data)

## engineDisplacement - объем двигателя

In [ ]:
compare_features_in_df(df_train, df_test, 'engineDisplacement', show_report=True)

Видим, что в тестовой выборке к значениям добавлено LTR, а в тренировочной обозначение дизеля. Приведем признак к одинаковому виду.

In [ ]:
# Отсчем LTR и d
df_test.engineDisplacement = df_test.engineDisplacement.apply(lambda x: x.split()[0])
df_train.engineDisplacement = df_train.engineDisplacement.apply(lambda x: x.rstrip('d'))


In [ ]:
compare_features_in_df(df_train, df_test, 'engineDisplacement', show_report=True)

Видно, что в тренировочной выборке больше уникальных значений объема двигателя. Вероятно лишние значения мы отсечем в будущем, а пока добавим признак как числовой (чем больше объем, тем дороже автомобиль). Однако, в признаке присутствуют строковые значения Electro и LTR и это нужно обработать. Чтобы особо не мучаться, добавим новый бинарный признак is_electro_car, а объем оставим 0.

In [ ]:
df_train['is_electro_car'] = ((df_train.engineDisplacement == 'Electro') | (df_train.engineDisplacement == 'LTR')).astype('int')
df_test['is_electro_car'] = (df_test.engineDisplacement == 'LTR').astype('int')

In [ ]:
# Заменим значения LTR и Electrocar на 0
df_train.engineDisplacement = df_train.engineDisplacement.replace({'Electro': 0})
df_test.engineDisplacement = df_test.engineDisplacement.replace({'LTR': 0})

In [ ]:
# Приведем в числовой вид
df_train.engineDisplacement = df_train.engineDisplacement.astype('float64')
df_test.engineDisplacement = df_test.engineDisplacement.astype('float64')

In [ ]:
# Посмотрим распределение признака
plot_dist_log(df_train, 'engineDisplacement', figsize=FIGSIZE)
plot_dist_log(df_test, 'engineDisplacement', figsize=FIGSIZE)

Распределение логнормальное. Добавим новый признак в датасет.

In [ ]:
# Создадим новый логарифмированный признак
df_train['engineDisplacement_log'] = df_train['engineDisplacement'].apply(lambda x: np.log(x+1))
df_test['engineDisplacement_log'] = df_test['engineDisplacement'].apply(lambda x: np.log(x+1))

## enginePower

In [ ]:
compare_features_in_df(df_train, df_test, 'enginePower', show_report=True)

Опять наши значения не пересекаются в двух выборках. Нужно это поправить. Удалим из тестовой выборки текст **N12**.

In [ ]:
df_test.enginePower = df_test.enginePower.apply(lambda x: x.rstrip('N12'))

In [ ]:
compare_features_in_df(df_train, df_test, 'enginePower', show_report=True)

In [ ]:
# Приведем в числовой вид
df_train.enginePower = df_train.enginePower.astype('int64')
df_test.enginePower = df_test.enginePower.astype('int64')

In [ ]:
# Посмотрим распределение признака
plot_dist_log(df_train, 'enginePower', figsize=FIGSIZE)
plot_dist_log(df_test, 'enginePower', figsize=FIGSIZE)

In [ ]:
# Создадим новый логарифмированный признак
df_train['enginePower_log'] = np.log(df_train.enginePower + 1)
df_test['enginePower_log'] = np.log(df_test.enginePower + 1)

## equipmentDict - словарь с опциями

In [ ]:
df_train.head(2)

In [ ]:
df_train.equipment_dict.value_counts().nlargest(5)

Видно, что словарь содержит достаточно много пропусков - **24437**. Можно попробовать вытащить из описания какие-то признаки, а из словаря сделать дамми-переменные. На данном этапе оставим как есть. Поработаем с данными признаками после EDA.

In [ ]:
# Посмотрим на список уникальных опций
data = merge_dataset(df_train, df_test)
data.equipment_dict = data.equipment_dict.fillna('{"nan": True}')

Получим уникальные значения опций и проверим предположение, что значение ключа опций содержит только True, т.е. нам нужны только ключи словаря

In [ ]:
unique_options = data.equipment_dict.to_dict()
unique_vals = set()
import ast

false_count = 0
for item in unique_options.values():
    item = item.replace('true', "True")
    opt = ast.literal_eval(item)
    for key, value in opt.items():
        # Убедимся, что оптиции не содержат False
        if value == False:
            false_count += 1
        unique_vals.add(key)

### Оценим количество опций

In [ ]:
# Количество опций из которых сделаем дамми переменные
print(f'Уникальных опций: {len(unique_vals)}')
print(unique_vals)

Видно, что много непонятных опций. Попробуем отфильтровать лишние и оставить наиболее понятные.

In [ ]:
import re

regex = r"([0-9A-Z]){3,}\b"
test_str = ' '.join(unique_vals)
replaced_str = re.sub(regex, '', test_str, 0, re.MULTILINE)
filtered_options = replaced_str.split()

len(filtered_options)

In [ ]:
# Приведем опции к виду словаря
data.equipment_dict = data.equipment_dict.apply(lambda s: s.replace('true', 'True'))
data.equipment_dict = data.equipment_dict.map(ast.literal_eval)

In [ ]:
# Приведем словарь к списку опций
data.equipment_dict = data.equipment_dict.apply(lambda s: [option for option, value in s.items() if option in filtered_options])

In [ ]:
# Посмотрим на самые популярные опции
from collections import Counter

counter = Counter()
data.equipment_dict.apply(lambda x: counter.update(x))

counter.most_common(20)

## fuelType - вид топлива

In [ ]:
compare_features_in_df(df_train, df_test, 'fuelType', show_report=True)

Тут все просто - создаем словарь соответствия и заменям значения в трейне.

In [ ]:
gasoline = {
    'GASOLINE': 'бензин',
    'DIESEL': 'дизель',
    'HYBRID': 'гибрид',
    'ELECTRO': 'электро',
    'LPG': 'газ'
}

df_train.fuelType = df_train.fuelType.replace(gasoline)

In [ ]:
compare_features_in_df(df_train, df_test, 'fuelType', show_report=True)

На данном этапе работу с признаком закончили.

## mileage - пробег

In [ ]:
compare_features_in_df(df_train, df_test, 'mileage', show_report=True)

In [ ]:
plot_dist_log(df_train, 'mileage', figsize=FIGSIZE)
plot_dist_log(df_test, 'mileage', figsize=FIGSIZE)

Как видим распределение с правым хвостом, при этом имеются выбросы в нулевой зоне (в выборке есть новые автомобили). Выбросы будем обрабатывать позже.

## modelDate - год выпуска модели

In [ ]:
compare_features_in_df(df_train, df_test, 'modelDate', show_report=True)

In [ ]:
plot_dist_log(df_train, 'modelDate', figsize=FIGSIZE)
plot_dist_log(df_test, 'modelDate', figsize=FIGSIZE)

Видим, что хвост смещен влево, так как в данных присутствуют раритеные автомобили. Оставим на этап работы с выбросами. Сразу создадим признак **modelAge** - сколько модели лет.

In [ ]:
df_train['modelAge'] = CURRENT_YEAR - df_train['modelDate']
df_test['modelAge'] = CURRENT_YEAR - df_test['modelDate']


## model_info - сборный признак информации по модели

In [ ]:
compare_features_in_df(df_train, df_test, 'model_info', show_report=True)

Ничего интересного - просто словарь некоторых параметров, которые у нас и так есть. **ВЫВОД:** в утиль.

In [ ]:
## model_name - название модели

In [ ]:
compare_features_in_df(df_train, df_test, 'model_name', show_report=True)

Много уникальных значений в тренировочной выборке, т.к. в тестовой всего 12 брэндов. Почти полное пересечение с тренировочной выборкой. Оставим признак как категориальный.

## name - сборное название модели

In [ ]:
compare_features_in_df(df_train, df_test, 'name', show_report=True)

Составной признак - содержит информацию, которая у нас есть в других колонках (объем двигателя, коробка передач и мощность двигателя). Посмотрим как поведет себя на обучении.

## numberOfDoors - количество дверей

In [ ]:
compare_features_in_df(df_train, df_test, 'numberOfDoors', show_report=True)

Впринципе тут всё предсказуемо. Однако машины с 0 дверьми - это весьма интересно :) Давайте это поправим.

In [ ]:
df_train[df_train.numberOfDoors == 0]

Видно, что это редкие гоночные болиды и вездеходы. Для нашей выборки это выбросы. С ними разберемся отдельно.

## productionDate - год выпуска автомобиля

In [ ]:
compare_features_in_df(df_train, df_test, 'productionDate', show_report=True)

Видим, что по году выпуска все ровно - не пересекается всего 4 года. Посмотрим на распределение признака.

In [ ]:
plot_dist_log(df_train, 'productionDate', figsize=FIGSIZE)
plot_dist_log(df_test, 'productionDate', figsize=FIGSIZE)

Видно, что признак очень похож на modelDate и вероятнее всего у них будет высокая корреляция. Это мы проверим, когда будем строить heatmap.

# sell_id, super_gen

In [ ]:
compare_features_in_df(df_train, df_test, 'sell_id', show_report=True)

**sell_id** - Просто инсайтище в данных :) Видно, что это уникальный идентификатор объявления на auto.ru и мы можем по нему определить 19367 объявлений по трейну, но по правилам соревнования это запрещено, поэтому признак в будущем удалим.

In [ ]:
df_test.super_gen.value_counts().nlargest(3)

**super_gen** - содержит словарь с технической информацией автомобиля. При беглом осмотре видим, что из него можно достать новые признаки - ускорение до 100км/ч, расход топлива и дорожный просвет. 

In [ ]:
# Переведем в словарь
data = merge_dataset(df_train, df_test)
#data.super_gen = data.super_gen.map(ast.literal_eval)

In [ ]:
# Вытащим новые признаки из super_gen
def super_gen_extract_feature(s, feature):
    if feature in s.keys():
        return s[feature]
    else:
        return np.nan

# Фичи из super-gen
data['acceleration'] = data['super_gen'].apply(super_gen_extract_feature, args=('acceleration',))
data['clearance_min'] = data['super_gen'].apply(super_gen_extract_feature, args=('clearance_min',))
data['fuel_rate'] = data['super_gen'].apply(super_gen_extract_feature, args=('fuel_rate',))


In [ ]:
# Посмотрим сколько пропусков
print('Acceleration: ', data.acceleration.isnull().sum())
print('Clearence', data.clearance_min.isnull().sum())
print('FuelRate', data.fuel_rate.isnull().sum())

Есть достаточно пропусков, но заполнить их по другим данным не проблема.

In [ ]:
# # Заполним средним по типу кузова
# display(data.groupby('bodyType')['acceleration'].mean().round(1))

# # Есть пропуски, нужно их заполнить тоже :) Посмотрим среднее ускорение по объему двигателя
# display(data[data.bodyType.isin(['седан-хардтоп', 'фастбек'])].enginePower.mean())

# ep_grp = data.groupby('enginePower').acceleration.mean()
# ep_grp[(ep_grp.index > 130) & (ep_grp.index < 135)]

# # Создаем словарь, а наши пропуски по кузову заполняем средним в 12 секунд
# mean_acceleration = data.groupby('bodyType')['acceleration'].mean().round(1).to_dict()

# print(mean_acceleration)

def fill_super_gen(s, feature):
    assert (feature in ['acceleration', 'fuel_rate', 'clearance_min'])
    
    replaces = {
        'acceleration': {'внедорожник': 9.9, 'кабриолет': 7.6, 'компактвэн': 13.0, 'купе': 6.7, 'купе-хардтоп': 7.2,
                             'лимузин': 7.5, 'лифтбек': 10.1, 'микровэн': 18.0, 'минивэн': 12.8,
                             'пикап': 13.5, 'родстер': 6.7, 'седан': 10.2, 'седан-хардтоп': 12,
                             'спидстер': 2.4, 'тарга': 8.6, 'универсал': 11.4, 'фастбек': 12,
                             'фургон': 18.4, 'хэтчбек': 12.0},

        'fuel_rate': {'внедорожник': 9.0, 'кабриолет': 9.4, 'компактвэн': 7.2, 'купе': 8.9, 'купе-хардтоп': 8.9,
                         'лимузин': 14.9, 'лифтбек': 6.7, 'микровэн': 5.6, 'минивэн': 8.4, 'пикап': 9.1, 'родстер': 9.8,
                         'седан': 7.7, 'седан-хардтоп': 9.3, 'спидстер': 7.6, 'тарга': 9.6, 'универсал': 7.7, 'фастбек': 9.5,
                         'фургон': 8.6, 'хэтчбек': 6.8},

        'clearance_min': {'внедорожник': 203.0, 'кабриолет': 135.0, 'компактвэн': 151.0, 'купе': 131.0, 'купе-хардтоп': 153.0,
                          'лимузин': 154.0, 'лифтбек': 155.0, 'микровэн': 160.0, 'минивэн': 168.0, 'пикап': 216.0, 'родстер': 134.0,
                          'седан': 151.0, 'седан-хардтоп': 151.0, 'спидстер': 50, 'тарга': 129.0, 'универсал': 154.0,
                          'фастбек': 135.0, 'фургон': 157.0, 'хэтчбек': 151.0}
    }
    
    return replaces[feature][s]

data['acceleration'] = data.apply(lambda row: fill_super_gen(row.bodyType, 'acceleration') if pd.isnull(row.acceleration) else row.acceleration, axis=1)
data['clearance_min'] = data.apply(lambda row: fill_super_gen(row.bodyType, 'clearance_min') if pd.isnull(row.clearance_min) else row.clearance_min, axis=1)
data['fuel_rate'] = data.apply(lambda row: fill_super_gen(row.bodyType, 'fuel_rate') if pd.isnull(row.fuel_rate) else row.fuel_rate, axis=1)



print('Acceleration: ', data.acceleration.isnull().sum())
print('Clearence', data.clearance_min.isnull().sum())
print('FuelRate', data.fuel_rate.isnull().sum())

## vehicleConfiguration - конфигурация авто

In [ ]:
df_train, df_test = data_split(data)
df_train.vehicleConfiguration.value_counts()

Признак является составным из bodyType, vehicleTransmission и engineDisplacement. Смело удаляем его.

## vehicleTransmission - коробка передач

In [ ]:
compare_features_in_df(df_train, df_test, 'vehicleTransmission', show_report=True)

Полное соответствие признака - просто приведем его к единому виду.

In [ ]:
transmission = {
    'AUTOMATIC': 'автоматическая',
    'MECHANICAL': 'механическая',
    'ROBOT': 'вариатор',
    'VARIATOR': 'роботизированная'
}

df_train.vehicleTransmission = df_train.vehicleTransmission.replace(transmission)

## vendor - тип производитея авто

In [ ]:
compare_features_in_df(df_train, df_test, 'vendor', show_report=True)

Видим, что в тестовой выборке присутствуют только японские и европейские марки, поэтому посмотрим, что лучше сработает - удаление лишних или их группировка в отдельную категорию - OTHER.

## Владельцы

In [ ]:
compare_features_in_df(df_train, df_test, 'Владельцы', show_report=True)

В тренировочной выборке присутствуют пропуски. Посмотрим на их количество. Предполагаю, что это новые автомобили.

In [ ]:
df_train['Владельцы'].isnull().sum()

In [ ]:
# Посмотрим на значения пробега
df_train[df_train['Владельцы'].isnull()].mileage.value_counts()

Как мы и предполагали - это объявления о продаже новых автомобилей. Выделим их в отдельную - 0, а там посмотрим на этапе работы с выбросами.

In [ ]:
# Приведем признак в соответствие
owners = {
    1.0: '1 владелец',
    2.0: '2 владельца',
    3.0: '3 или более'

}

df_train['Владельцы'] = df_train['Владельцы'].replace(owners)
df_train['Владельцы'] = df_train['Владельцы'].fillna('новый авто')

In [ ]:
compare_features_in_df(df_train, df_test, 'Владельцы', show_report=False)

Наши четыре категории приведены к единому виду. 

## Владение

In [ ]:
compare_features_in_df(df_train, df_test, 'Владение', show_report=False)

Помним при беглом осмотре данных, что признак содержит очень много пропусков. Убедимся в этом еще раз.

In [ ]:
eda_checks(df_train)
eda_checks(df_test)

72% пропусков на трейне и 65% на тесте. Однозначно удаляем признак.

## ПТС

In [ ]:
compare_features_in_df(df_train, df_test, 'ПТС', show_report=True)

Признак содержит два значения и пропуски. Во-первых, приведем признак к единому виду, а во вторых заполним пропуски как Дубликат.

In [ ]:
pts = {
    True: 'Оригинал',
    np.nan: 'Дубликат'
}

df_train['ПТС'] = df_train['ПТС'].replace(pts)
df_test['ПТС'] = df_test['ПТС'].replace(pts)


In [ ]:
compare_features_in_df(df_train, df_test, 'ПТС', show_report=False)

## Привод

In [ ]:
compare_features_in_df(df_train, df_test, 'Привод', show_report=True)

Категориальный признак на 3 категории. Просто приведем к единому виду.

In [ ]:
gear_type = {
    'FORWARD_CONTROL': 'передний',
    'ALL_WHEEL_DRIVE': 'полный',
    'REAR_DRIVE': 'задний'
}

df_train['Привод'] = df_train['Привод'].replace(gear_type)

In [ ]:
compare_features_in_df(df_train, df_test, 'Привод', show_report=False)

## Руль

In [ ]:
compare_features_in_df(df_train, df_test, 'Руль', show_report=False)

Тут все просто - приводим к единому виду.

In [ ]:
wheel_drive = {
    'LEFT': 'Левый',
    'RIGHT': 'Правый'
}

df_train['Руль'] = df_train['Руль'].replace(wheel_drive)

In [ ]:
compare_features_in_df(df_train, df_test, 'Руль', show_report=False)

## Состояние

In [ ]:
compare_features_in_df(df_train, df_test, 'Состояние', show_report=True)

Бесполезный признак с 1 значением - на удаление.

## Таможня

In [ ]:
compare_features_in_df(df_train, df_test, 'Таможня', show_report=True)

Тоже признак с 1 значением - на удаление.

## price - цена авто - наш таргет!

In [ ]:
data = merge_dataset(df_train, df_test)

# Посмотрим распределение цены по брэндам
df_comp_avg_price = plot_feature_by_avg_target(data, 'brand', plot=False)

In [ ]:
# Разделим авто по категориям по средней цене
data = data.merge(df_comp_avg_price, on = 'brand')
data['brand_category'] = data['brand_avg_price'].apply(lambda x : "Budget" if x < 0.1e7 
                                                     else ("Mid_Range" if 0.1e7 <= x < 0.3e7
                                                           else "Luxury"))

In [ ]:
# По типу топлива
df_fuel_avg_price = plot_feature_by_avg_target(data, 'fuelType')

Электрокары в выборке самые дорогие.

In [ ]:
# По типу корпуса
df_bodyType_avg_price = plot_feature_by_avg_target(data, 'bodyType')

Самая высокая цена у лимузинов, спидстеров и тарга

In [ ]:
# По дверям
df_doors_avg_price = plot_feature_by_avg_target(data, 'numberOfDoors')

Самые дорогие авто без дверей - спидстеры, что логично

In [ ]:
df_geartype_avg_price = plot_feature_by_avg_target(data, 'Привод')

In [ ]:
plot_dist_log(df_train, 'price', figsize=FIGSIZE)

Видим, что распределение логнормальное. Посмотрим на распределение с другими признаками.

In [ ]:
df_train['price_log'] = df_train.price.map(np.log1p)
cols = ['engineDisplacement', 'enginePower', 'mileage', 'modelAge', 'modelDate', 'productionDate']
sns.pairplot(df_train, x_vars=cols, y_vars=['price_log'], kind='reg')

In [ ]:
eda_checks(df_train)

Есть пропуски в таргете. Просто удалим эти записи.

In [ ]:
df_train = df_train[~df_train.price.isnull()]

In [ ]:
df_train.shape

## Визуализация Price с числовыми признаками

In [ ]:
data = merge_dataset(df_train, df_test)
num_cols = data.select_dtypes(include=['int64', 'float64']).columns

print(num_cols)

In [ ]:
sns.scatterplot(x="engineDisplacement", y="price_log", data=data,color='purple')

In [ ]:
sns.scatterplot(x="enginePower", y="price_log", data=data,color='purple')

In [ ]:
sns.scatterplot(x="mileage", y="price_log", data=data,color='purple')

In [ ]:
sns.scatterplot(x="modelDate", y="price_log", data=data,color='purple')

## Корреляция

In [ ]:
heatmap_numeric_target_variable(data, 'price_log')

In [ ]:
plot_correlation(data.select_dtypes(include=['int64', 'float64']).corr())

Как видим, больше всего взаимосвязь таргета с мощностью двигателя, датой производства и выпуска модели, объемом двигателя и пробегом автомобиля. Между собой высокая корреляция у enginePower и engineDisplacement и modelDate и productionDate. Однозначно признаки Date нужно будет удалять после преобразования в новые.

## Значимость признаков

In [ ]:
# ANOVA
plot_fclassif(df_train, cols, 'price')

In [ ]:
# MUTUAL

# Удалим лишние признаки
df_ = df_train.drop(['super_gen', 'equipment_dict', 'model_info', 'complectation_dict', 'car_url', 'description', 'image', 'Владение'], axis=1)

# Отберем только категориальные
df_ = df_.select_dtypes(include=['object'])

# Нужно закодировать признаки для функции mutual_info_classif
le = OrdinalEncoder()
df_encoded_values = le.fit_transform(df_)

# Признаки для теста
test_cols = list(df_.columns)

# np.array закодированных признаков и таргета
df_encoded_values = np.c_[df_encoded_values, df_train.price.values.reshape(-1, 1)]

# Заворачиваем в датафрэйм
df_encoded = pd.DataFrame(df_encoded_values, columns=test_cols + ['price'])

# Смотрим статистически значимые признаки
plot_mutual_info_classif(df_encoded, df_.columns, 'price')


**EDA ВЫВОДЫ:**
- {C} **bodyType**: Привели признак в соответствие выборок. Определили как категориальный.    

- {C} **brand**: В тестовой выборке 12 марок автомобилей, в обучаюшей 121. Привели марку Mercedes в единый формат между выборками. С группировкой или удалением данных поиграемся на этапе ML. Определили как категориальный.  

- ~~**car_url**~~ - ссылка на объявление. Удаляем признак. 

- {C} **color** - цвет автомобилей привели в соответствие. Определили как категориальный.

- ~~**complectation_dict**~~ - много пропусков. Удаляем признак.

- **description** - на этапе feauture engineering, возможно достанем полезную информацию.

- {N} **engineDisplacement** - Привели признак в соответствие выборок. Автомобили с электродвигателем отметили как имеющие объем 0 и дополнительно введем признак is_electro_car на этапе FE. Определили как числовой.

- {N} **enginePower** - Привели в соответствие. Определили как числовой.         

- **equipment_dict** - Из признака можно достать много dummy-переменных. Оставили на этап FE.
      
- {C} **fuelType** - Привели в соответствие выборок. Определили как категориальный.

- ~~**image**~~ - ссылки на фото. Удаляем.       

- {N} **mileage** - Пробег автомобиля имеет выбросы в точке 0 (новые авто). Определили как числовой.

- {N} **modelDate** - Дата производства марки. Коррелирует с датой производства автомобиля. Числовой.   

- ~~**model_info**~~ - Не содержит полезной информации. Удаляем.       

- {C} **model_name** - Модель авто. Категориальный.  

- {C} **name** - Составной признак. Посмотрим как поведет себя при обучении. Возможно удалим.   

- {C} **numberOfDoors** - Привели в соответствие выборок. Категориальный.      

- ~~**parsing_unixtime**~~ - время парсинга. Удаляем.   

- ~~**priceCurrency**~~ - валюта. Удаляем.   

- {N} **productionDate** - Дата выпуска модели. Создадим новый признак на этапе FE.    

- ~~**sell_id**~~ - ИНСАЙТ! ID объявления с авто.ру. Удаляем в соответствие с правилами.

- **super_gen** - содержит словарь с технической информацией автомобиля. При беглом осмотре видим, что из него можно достать новые признаки - ускорение до 100км/ч, расход топлива и дорожный просвет.       

- {C} **vehicleConfiguration** - Составной признак. Посмотрим как поведет себя.

- {C} **vehicleTransmission** - привели в соответствие выборок. Категориальный. 

- {C} **vendor** - В тестовой выборке присутствуют только японские и европейские марки, поэтому посмотрим, что лучше сработает - удаление лишних или их группировка в отдельную категорию - OTHER. Категориальный. 

- {C} **Владельцы** - привели в соответствие выборок. Пропуски определили как новый автомобиль. Категориальный. 

- ~~**Владение**~~ - Много пропусков в данных. Удаляем.       

- {C} **ПТС** - Определили пропуски как дупикат ПТС. Категориальный.    

- {C} **Привод** - Привели в соответствие выборок. Категориальный.  

- {C}{B} **Руль** - привелив соответствие. Бинарный или категориальный.   
             
- ~~**Состояние**~~ - Одно значение. Удаляем.   

- ~~**Таможня**~~ - Одно значение. Удаляем.    
         
- {T} **price** - Наш таргет. Наибольшая связь с мощностью двигателя, датой производства и выпуска модели, объемом двигателя и пробегом автомобиля.    

Подводя итоги, у нас получается для работы есть 14 категориальных признаков, 5 сисловых, 3 для создания новых признаков.  

![Очистка данных](https://ua.all.biz/img/ua/service_catalog/432798.jpeg)
# Data Preprocessing
### На данном этапе завернем всё, что мы обнаружили в рамках EDA при помощи SKlearn API для использования в пайплайне

In [ ]:
# Напишем наши классы обработки данных
class DatasetProcessing(BaseEstimator, TransformerMixin):
    def __init__(self, use_log=False, cat_boost_prepare=False, filter_data=False, target_encode=False):
        self.use_log=use_log
        self.cat_boost_prepare = cat_boost_prepare
        self.filter_data = filter_data
        self.target_encode = target_encode
        
        self.test_brands = ['BMW',           
                            'VOLKSWAGEN',   
                            'NISSAN', 
                            'MERCEDES',     
                            'TOYOTA',       
                            'AUDI',         
                            'MITSUBISHI',   
                            'SKODA',        
                            'VOLVO',        
                            'HONDA',        
                            'INFINITI',     
                            'LEXUS'
                           ]
        
        # Наши словари замен
        self.dicts = {
            'brand': {'MERCEDES-BENZ':'MERCEDES'},
            'color':  {
                        'EE1D19': 'красный',
                        'FFD600': 'жёлтый',
                        'DEA522': 'золотистый',
                        'FAFBFB': 'белый',
                        'CACECB': 'серебристый',
                        '040001': 'чёрный',
                        '0000CC': 'синий',
                        '97948F': 'серый',
                        '200204': 'коричневый',
                        '007F00': 'зелёный',
                        '660099': 'пурпурный',
                        'FF8649': 'оранжевый',
                        '22A0F8': 'голубой',
                        'C49648': 'бежевый',
                        '4A2197': 'фиолетовый',
                        'FFC0CB': 'розовый',
                        '40001': 'чёрный'
                    },
             'engineDisplacement': {
                        'LTR': 0,
                        'Electro': 0,
             },

             'fuelType': {
                        'GASOLINE': 'бензин',
                        'DIESEL': 'дизель',
                        'HYBRID': 'гибрид',
                        'ELECTRO': 'электро',
                        'LPG': 'газ'
            },
            'transmission': {
                        'AUTOMATIC': 'автоматическая',
                        'MECHANICAL': 'механическая',
                        'ROBOT': 'вариатор',
                        'VARIATOR': 'роботизированная'
            },
            'owners':  {
                        1.0: '1 владелец',
                        2.0: '2 владельца',
                        3.0: '3 или более'

            },
            'pts': {
                        True: 'Оригинал',
                        np.nan: 'Дубликат'
            },
            'gear_type': {
                        'FORWARD_CONTROL': 'передний',
                        'ALL_WHEEL_DRIVE': 'полный',
                        'REAR_DRIVE': 'задний'
            },
            'wheel_drive': {
                        'LEFT': 'Левый',
                        'RIGHT': 'Правый'
            }


        }

    def get_dicts(self, feature):
        pass

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        X_ = X.copy() # работаем с копией
        
        # Группируем данные по брэнду        
        if self.filter_data:
            brands = list(X_[~X_.brand.isin(self.test_brands)].brand.value_counts().index)
            X_.brand = X_.brand.apply(lambda x: 'OTHER' if x in brands else x)

        # bodyType
        X_.dropna(subset=['bodyType'], inplace=True)
        X_['bodyType'] = X_['bodyType'].apply(lambda x: x.split()[0])
        
        # brand
        X_.brand = X_.brand.replace(self.dicts['brand'])

        # color
        X_.color = X_.color.replace(self.dicts['color'])

        # engineDisplacement
        X_.engineDisplacement = X_.engineDisplacement.apply(lambda x: x.split()[0])
        X_.engineDisplacement = X_.engineDisplacement.apply(lambda x: x.rstrip('d'))
        X_['is_electro_car'] = ((X_.engineDisplacement == 'Electro') | (X_.engineDisplacement == 'LTR')).astype('int')
        X_.engineDisplacement = X_.engineDisplacement.replace(self.dicts['engineDisplacement'])
        X_.engineDisplacement = X_.engineDisplacement.astype('float64')

        # enginePower
        X_.enginePower = X_.enginePower.astype('str')
        X_.enginePower = X_.enginePower.apply(lambda x: x.rstrip('N12'))
        X_.enginePower = X_.enginePower.astype('float64')

        # FuelType
        X_.fuelType = X_.fuelType.replace(self.dicts['fuelType'])

        # vehicleTransmission
        X_.vehicleTransmission = X_.vehicleTransmission.replace(self.dicts['transmission'])

        # Владельцы
        X_['Владельцы'] = X_['Владельцы'].replace(self.dicts['owners'])
        X_['Владельцы'] = X_['Владельцы'].fillna('новый авто')

        # ПТС
        X_['ПТС'] = X_['ПТС'].replace(self.dicts['pts'])

        # Привод
        X_['Привод'] = X_['Привод'].replace(self.dicts['gear_type'])

        # Руль
        X_['Руль'] = X_['Руль'].replace(self.dicts['wheel_drive'])

        # Desription
        X_.description = X_.description.fillna('[]')

        # Пропуски в Price
        if 'price' in X_.columns:
            X_test = X_[X_['sample'] == 0]
            X_train = X_[X_['sample'] == 1]
            X_train = X_train[~X_train.price.isnull()]
            X_ = X_test.append(X_train, sort=False).reset_index(drop=True) # объединяем


        # Пропуски в equipment_dict
        X_.equipment_dict = X_.equipment_dict.fillna('{}')
        
#         # Brand
#         df_comp_avg_price = plot_feature_by_avg_target(X_, 'brand', plot=False)
#         X_ = X_.merge(df_comp_avg_price, on = 'brand')
#         X_['brand_category'] = X_['brand_avg_price'].apply(lambda x : "Budget" if x < 0.1e7 
#                                                      else ("Mid_Range" if 0.1e7 <= x < 0.3e7
#                                                            else "Luxury"))

                # superGen
        # Вытащим новые признаки из super_gen
        def super_gen_extract_feature(s, feature):
            if feature in s.keys():
                return s[feature]
            else:
                return np.nan

        # Фичи из super-gen
        X_['acceleration'] = X_['super_gen'].apply(super_gen_extract_feature, args=('acceleration',))
        X_['clearance_min'] = X_['super_gen'].apply(super_gen_extract_feature, args=('clearance_min',))
        X_['fuel_rate'] = X_['super_gen'].apply(super_gen_extract_feature, args=('fuel_rate',))
        
        # super_gen
        def fill_super_gen(s, feature):
            assert (feature in ['acceleration', 'fuel_rate', 'clearance_min'])

            replaces = {
                'acceleration': {'внедорожник': 9.9, 'кабриолет': 7.6, 'компактвэн': 13.0, 'купе': 6.7, 'купе-хардтоп': 7.2,
                                     'лимузин': 7.5, 'лифтбек': 10.1, 'микровэн': 18.0, 'минивэн': 12.8,
                                     'пикап': 13.5, 'родстер': 6.7, 'седан': 10.2, 'седан-хардтоп': 12,
                                     'спидстер': 2.4, 'тарга': 8.6, 'универсал': 11.4, 'фастбек': 12,
                                     'фургон': 18.4, 'хэтчбек': 12.0},

                'fuel_rate': {'внедорожник': 9.0, 'кабриолет': 9.4, 'компактвэн': 7.2, 'купе': 8.9, 'купе-хардтоп': 8.9,
                                 'лимузин': 14.9, 'лифтбек': 6.7, 'микровэн': 5.6, 'минивэн': 8.4, 'пикап': 9.1, 'родстер': 9.8,
                                 'седан': 7.7, 'седан-хардтоп': 9.3, 'спидстер': 7.6, 'тарга': 9.6, 'универсал': 7.7, 'фастбек': 9.5,
                                 'фургон': 8.6, 'хэтчбек': 6.8},

                'clearance_min': {'внедорожник': 203.0, 'кабриолет': 135.0, 'компактвэн': 151.0, 'купе': 131.0, 'купе-хардтоп': 153.0,
                                  'лимузин': 154.0, 'лифтбек': 155.0, 'микровэн': 160.0, 'минивэн': 168.0, 'пикап': 216.0, 'родстер': 134.0,
                                  'седан': 151.0, 'седан-хардтоп': 151.0, 'спидстер': 50, 'тарга': 129.0, 'универсал': 154.0,
                                  'фастбек': 135.0, 'фургон': 157.0, 'хэтчбек': 151.0}
            }

            return replaces[feature][s]

        X_['acceleration'] = X_.apply(lambda row: fill_super_gen(row.bodyType, 'acceleration') if pd.isnull(row.acceleration) else row.acceleration, axis=1)
        X_['clearance_min'] = X_.apply(lambda row: fill_super_gen(row.bodyType, 'clearance_min') if pd.isnull(row.clearance_min) else row.clearance_min, axis=1)
        X_['fuel_rate'] = X_.apply(lambda row: fill_super_gen(row.bodyType, 'fuel_rate') if pd.isnull(row.fuel_rate) else row.fuel_rate, axis=1)

        # CatBoost LabelEncoder
        if self.cat_boost_prepare:
            for column in X_.select_dtypes(include=['object']).columns:
                if column not in ['description', 'equipment_dict', 'super_gen']:
                    X_[column] = X_[column].astype('category').cat.codes
            
        # Приведем числовые признаки к единому виду
        X_['enginePower'] =  X_['enginePower'].astype('int64')
        X_['modelDate'] =  X_['modelDate'].astype('int64')
        X_['productionDate'] = X_['productionDate'].astype('int64')
        X_['numberOfDoors'] = X_['numberOfDoors'].astype('int64')

        # Производим логарифмирование признаков
        if self.use_log:
            X_['engineDisplacement_log'] = np.log(X_['engineDisplacement'] + 1)
            X_['enginePower_log'] = np.log(X_.enginePower + 1)
            X_['mileage_log'] = np.log1p(X_.mileage)
            # X_.drop(['engineDisplacement', 'enginePower', 'mileage'], axis=1, inplace=True)
            
        # Rename columns
        columns_rename = {'Владельцы': 'owners', 'Владение': 'ownage', 'ПТС': 'pts', 'Привод': 'gear_type', 'Руль': 'wheeldrive', 'Состояние': 'health', 'Таможня': 'custom'}                
        X_.rename(columns=columns_rename, inplace=True)

        return X_

# FeatureEngineering
class DatasetFE(BaseEstimator, TransformerMixin):
    def __init__(self, ohe=False, use_equip=False):
        self.ohe=ohe
        self.use_equip=use_equip

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        import ast
        import re         
          
        if self.use_equip:
            # equipmentDict
            unique_options = X.equipment_dict.to_dict()
            unique_vals = set()
            for item in unique_options.values():
                item = item.replace('true', "True")
                opt = ast.literal_eval(item)
                for key, value in opt.items():
                    unique_vals.add(key)

            # Приведем словарь опций в отфильтрованный список
            X.equipment_dict = X.equipment_dict.fillna('{"nan": True}')
            X.equipment_dict = X.equipment_dict.apply(lambda s: s.replace('true', 'True'))
            X.equipment_dict = X.equipment_dict.map(ast.literal_eval)
            X.equipment_dict = X.equipment_dict.apply(lambda s: [key for key, value in s.items() if key in unique_vals])
            equipmemt_bins = get_binary_dummies(X, 'equipment_dict')

            X = pd.concat([X, equipmemt_bins], axis=1)
        
        # model dates
        X['modelAge'] =  CURRENT_YEAR - X['modelDate']
        X['carAge'] = CURRENT_YEAR - X['productionDate']
        X['carAge_to_modelAge'] = X['modelAge'] - X['carAge']
        
        # milenage
        X['mileage_carAge'] = X['carAge'] / (X['mileage'] + 1)
        X['mileage_modelAge'] = X['modelAge'] / (X['mileage'] + 1)
        
        def permile(row):
            if not row['modelAge']:
                return row['mileage']
            else:
                return round(row['mileage'] / row['modelAge'], 1)

        def cat_mileagePerYear(x):
            if x < 15000: x = 1
            elif 15000 <= x < 30000: x = 2
            elif 30000 <= x < 45000: x = 3
            elif 45000 <= x: x = 4
            return x 
        
        def tax_engine_power(x):
            if x <= 100: x = 1
            elif 101 <= x <= 125: x = 2
            elif 126 <= x <= 150: x = 3
            elif 156 <= x <= 175: x = 4
            elif 176 <= x <= 200: x = 5
            elif 201 <= x <= 225: x = 6
            elif 226 <= x <= 250: x = 7
            elif 251 < x: x = 8
       
            return x
       
        def cat_mileage(x):
            if x < 25000: x = 1
            elif 25000 <= x < 50000: x = 2
            elif 50000 <= x < 75000: x = 3
            elif 75000 <= x < 100000: x = 4
            elif 100000 <= x < 125000: x = 5
            elif 125000 <= x < 150000: x = 6
            elif 150000 <= x < 175000: x = 7
            elif 175000 <= x < 200000: x = 8
            elif 200000 <= x < 225000: x = 9
            elif 225000 <= x < 250000: x = 10
            elif 250000 <= x < 275000: x = 11
            elif 275000 <= x < 300000: x = 12
            elif 300000 <= x < 325000: x = 13
            elif 325000 <= x < 350000: x = 14
            elif 350000 <= x < 375000: x = 15
            elif 375000 <= x < 400000: x = 16
            elif 400000 <= x: x = 17
                
            return x   
        
        X['cat_mileage'] = X['mileage'].apply(cat_mileage)
        X['tax_engine_power'] = X['enginePower'].apply(tax_engine_power)
        X['AgePerMile'] = X.apply(permile, axis=1)
        X['AgePerMile'] = X['AgePerMile'].astype('int64')
        X['cat_mileagePerYear'] = X['AgePerMile'].apply(lambda x: cat_mileagePerYear(x))
        
        # use OneHot
        if self.ohe:
            one_hot_cols = ['numberOfDoors', 'pts', 'wheeldrive', 'fuelType']
            X = pd.get_dummies(X, columns=one_hot_cols)   
               
        return X

# Отбираем фичи
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        self.features = list(features)

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        return X[self.features + ['sample', 'price']]
    

# Удаляем лишние фичи
class FeatureEraser(BaseEstimator, TransformerMixin):
    def __init__(self, features):
        assert(type(features) == list)
        self.features = features

    def fit(self, X, y = None):
        return self

    def transform(self, X, y = None):
        return X.drop(self.features, axis=1)
    
    
# TargetEncoder
class TargetEncoderWrapper(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns=None, target=None, df_type=None, train=None, n_folds=5):
        self.columns = columns
        self.target = target
        self.n_folds = n_folds
        self.df_type = df_type
        self.train = train
    
    def get_encoded_names(self, column):
        return column + '_' + 'Kfold_Target_Enc'
        
    def fit(self, X):
        return self
    
    def transform(self, X):
        assert(type(self.columns) == list)
        assert([col for col in self.columns if col in X.columns])
        assert(self.df_type in ['train', 'test'])
        
        for col in self.columns:
            if self.df_type == 'train':
                target_enc = KFoldTargetEncoderTrain(col, self.target, n_fold=self.n_folds)
                X = target_enc.fit_transform(X)
            else:
                assert([col for col in self.columns if col in self.train.columns])
                target_enc = KFoldTargetEncoderTest(self.train, col, self.get_encoded_names(col))
                X = target_enc.fit_transform(X)
            
        return X


# FEATURE SELECTION

In [ ]:
# load data
data = load_data()

# Плохие брэнды по результатам работы моделей
bad_mape_brands = [  'PROTON', 'HUANGHAI', 'TESLA', 'DW HOWER', 'DATSUN', 'FERRARI', 'MASERATI', 'IVECO',
                     'BAJAJ',
                     'ГОНОЧНЫЙ БОЛИД',
                     'СМЗ',
                     'DACIA',
                     'SHANGHAI MAPLE',
                     'DADI',
                     'ГАЗ',
                     'JMC',
                     'HAFEI',
                     'ЛУАЗ',
                     'OLDSMOBILE',
                     'МОСКВИЧ',
                     'AC',
                     'EXCALIBUR',
                     'EAGLE',
                     'ИЖ',
                     'MITSUOKA',
                     'ЗАЗ',
                     'METROCAB',
                     'DALLARA',
                     'GMC',
                     'FOTON',
                     'LINCOLN',
                     'HAWTAI',
                     'ZX',
                     'DONGFENG',
                     'AMC',
                     'TATRA',
                     'DS',
                     'PUCH',
                     'ISUZU',
                     'MAYBACH',
                     'PONTIAC',
                     'DERWAYS',
                     'ROVER',
                     'ASIA',
                     'HAIMA',
                     'MERCURY',
                     'ALFA ROMEO',
                     'BUICK',
                     'DAIHATSU',
                     'PLYMOUTH',
                     'BYD',
                     'MG',
                     'ASTON MARTIN',
                     'FIAT',
                     'SAAB',
                     'JAC',
                     'LADA (ВАЗ)',
                     'ARIEL',
                     'SATURN',
                     'DODGE',
                     'CHRYSLER',
                     'УАЗ',
                     'ТАГАЗ',
                     'BRILLIANCE',
                     'ЗИЛ',
                     'HUMMER',
                     'ALPINA',
                     'LANCIA',
                     'LAMBORGHINI',
                     'SCION',
                     'DAEWOO',
                     'TRIUMPH',
                     'VORTEX',
                     'FAW']

In [ ]:
# Отфильтруем из данных "плохие брэнды"
data = data[~data.brand.isin(bad_mape_brands)]

# И брэнды где записей меньше 100
data = data.groupby('brand').filter(lambda x: len(x) > 100)

In [ ]:
drop_cols = ['car_url', 'image', 'priceCurrency', 'ownage', 'sell_id', 'complectation_dict', 
                 'parsing_unixtime', 'model_info', 'health', 'custom', 'super_gen', 'description', 'equipment_dict']

# Catergory columns
cat_cols = ['model_name', 'name', 'brand', 'color', 'bodyType', 'gear_type',
                            'vehicleTransmission', 'vendor', 'vehicleConfiguration', 'owners',
                            'numberOfDoors', 'pts', 'wheeldrive', 'fuelType']


processing_pipe = make_pipeline(DatasetProcessing(cat_boost_prepare=True, use_log=True, target_encode=True),
                                DatasetFE(ohe=False, use_equip=True), 
                                FeatureGenerator(feature_list=['mileage', 'engineDisplacement_log', 'enginePower_log'],
                                                                            primitives='all'), FeatureEraser(drop_cols))

X, y, X_sub = prepare_data_pipeline(data, processing_pipe)

In [ ]:
# Сохраним незакодированные данные для анализа моделей
orig_cats = pd.DataFrame()
orig_cats = data.iloc[X.index, [0, 1, 3, 9, 14, 15, 16, 22, 23, 24, 25, 28]]

# # TargetEncode
target_encoder = ce.TargetEncoder(cols=cat_cols, smoothing=5).fit(X,y)
X = target_encoder.transform(X)
X_sub = target_encoder.transform(X_sub)


### Модель для отбора признаков

In [ ]:
# # Наша модель для отбора фич
# def cat_model(X_train, X_test, y_train, y_test):
#     model = CatBoostRegressor(iterations = 20000,
#                               learning_rate = 0.1,
#                               random_seed = RANDOM_SEED,
#                               eval_metric='MAPE',
#                               custom_metric=['R2', 'MAE'],
#                                od_type='Iter',
#                                od_wait=600,
#                               #max_ctr_complexity=1
#                              )

#     model.fit(X_train, y_train,
#              #cat_features=cat_features_ids,
#              eval_set=(X_test, y_test),
#              verbose_eval=500,
#              use_best_model=True,
#              plot=False,
#              )
    
#     return(model)

# subs_cat_fs, feature_importance_df, oof_fs = run_model_cv(cat_model, X=X, y=y, sub_test=X_sub, cv=5, 
#                            random_state=RANDOM_SEED, name='catboost_fe_fs', comment="")
def et_model(X_train, X_test, y_train, y_test):
    # Create a based model
    rf = ExtraTreesRegressor(n_jobs=-1, random_state=RANDOM_SEED, n_estimators=100)
    rf.fit(X_train, y_train)

    return (rf)

# # RFE FEATURE SELECTION
# select_estimator = ExtraTreesRegressor(n_jobs=-1, random_state=RANDOM_SEED, n_estimators=50)
# selector = RFE(select_estimator, n_features_to_select=100, step=10, verbose=1)
# selector = selector.fit(X, y)
# selector.support_

X.shape

In [ ]:
#rfe_cols = X.columns[selector.support_]

# FEATURE IMPORTANCE
subs_et_fs, feature_importance_df, oof_etfs_predicts = run_model_cv(et_model, X=X, y=y, sub_test=X_sub, cv=5, random_state=RANDOM_SEED, name='etree_default')

In [ ]:
pd.set_option('display.max_rows', 100)

# Ограничим количество фич
feature_limit = 100

# возьмем среднее значение важности фич из KFold
all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[:feature_limit]['feature'])
all_features[:feature_limit]

In [ ]:
# Проверим корреляцию фич
df = X[important_features]
corr_matrix = X.corr().abs()

# Верхний треугольник матрицы корреляций
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Индекс фич с корреляцией выше 0.95
high_cor = [column for column in upper.columns if any(upper[column] > 0.95)]
print(len(high_cor))
print(high_cor)

In [ ]:
# Удаляем сильно коррелирующией между собой фичи
features = [i for i in important_features if i not in high_cor]
print(len(features))
print(features)

# MODEL
### CatBoost Basemodel

In [ ]:
# Посмотрим как обучается базовая модель
X_train, X_test, y_train, y_test = train_test_split(X[features], y, test_size=VAL_SIZE, shuffle=True, random_state=RANDOM_SEED)

def cat_model(X_train, X_test, y_train, y_test):
    model = CatBoostRegressor(iterations = 20000,
                              learning_rate = 0.05,
                              random_seed = RANDOM_SEED,
                              eval_metric='MAPE',
                              custom_metric=['R2', 'MAE'],
                              od_type='Iter',
                              od_wait=150,
                             #'logging_level': 'Silent',
                              #max_ctr_complexity=1
                             )

    model.fit(X_train, y_train,
             #cat_features=cat_features_ids,
             eval_set=(X_test, y_test),
             verbose_eval=500,
             use_best_model=True,
             plot=False,
             )
    
    return(model)

#base_cat_model = cat_model(X_train, X_test, y_train, y_test)

In [ ]:
# CatBoostCV
sub_cat, feature_importances_df, oof_cat_predicts = run_model_cv(cat_model, X=X.loc[:, :], y=y, sub_test=X_sub.loc[:, :], cv=5, 
                           random_state=RANDOM_SEED, name='catboost_fe_11', comment="t_enc")

### Feature Importance SHAPley

In [ ]:
import shap

# Разобьтем трэйн
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, random_state=RANDOM_SEED)


model = cat_model(X_train, X_test, y_train, y_test)
shap_values = model.get_feature_importance(Pool(X_test, label=y_test), 
                                                                     type="ShapValues")


In [ ]:
expected_value = shap_values[0,-1]
shap_values_ = shap_values[:,:-1]

shap.initjs()
shap.force_plot(expected_value, shap_values_[77,:], X_test.iloc[77,:])

In [ ]:
shap.summary_plot(shap_values_, X_test)

### DTREE

In [ ]:
def dt_model(X_train, X_test, y_train, y_test):
    # Create a based model
    dt = DecisionTreeRegressor(random_state=RANDOM_SEED)
    dt.fit(X_train, y_train)

    return (dt)

base_dt_tree = dt_model(X_train, X_test, y_train, y_test)

In [ ]:
# Запускаем модель CV
sub_dt, fi, oof_df_predicts = run_model_cv(dt_model, X=X.loc[:, features], y=y, sub_test=X_sub.loc[:, features], cv=5, random_state=RANDOM_SEED, name='rf_cv')

## RandomForest

In [ ]:
grid_params = {  'max_depth': [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, None],
                 'max_features': ['auto', 'sqrt'],
                 'min_samples_leaf': [1, 2, 4],
                 'min_samples_split': [2, 5, 10],
                 'n_estimators': [10, 50, 100]}

def rf_model(X_train, X_test, y_train, y_test):
    # Create a based model
    rf = RandomForestRegressor(n_jobs=-1, random_state=RANDOM_SEED, n_estimators=100)
    rf.fit(X_train, y_train)

    return (rf)
    
#base_rf_model = rf_model(X_train, X_test, y_train, y_test)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)
rnd_clf = RandomizedSearchCV(RandomForestRegressor(), scoring=mape_scorer, cv=3, param_distributions=grid_params, n_jobs=-1, verbose=1)
rnd_clf.fit(X_train, y_train)
print(rnd_clf.best_params_)
print(f'Best score: {rnd_clf.best_score_}')

In [ ]:
rf = rnd_clf.best_estimator_
X_test.drop(['dummy_armored', 'dummy_U25'], axis=1, inplace=True)
X_train.drop(['dummy_armored', 'dummy_U25'], axis=1, inplace=True)
rf.fit(X_train, y_train)

In [ ]:
perm = PermutationImportance(rf, random_state=RANDOM_SEED).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
# LIME
explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns, class_names=['CarPrice'], verbose=True, mode='regression')
exp = explainer.explain_instance(X_test.values[27], rf.predict, num_features=10)
exp.show_in_notebook(show_table=True)

In [ ]:
y.iloc[X_test.iloc[27].name]

In [ ]:
# Запускаем модель CV
sub_rf, fi, oof_rf_predicts = run_model_cv(rf_model, X=X.loc[:, features], y=y, sub_test=X_sub.loc[:, features], cv=5, random_state=RANDOM_SEED, name='rf_cv', target_encoding=True, te_cols=cat_cols)


## ExtraTree Regressor

In [ ]:
def et_model(X_train, X_test, y_train, y_test):
    # Create a based model
    rf = ExtraTreesRegressor(n_jobs=-1, random_state=RANDOM_SEED, n_estimators=100)
    rf.fit(X_train, y_train)

    return (rf)

# Базовая модель
base_et_model = et_model(X_train, X_test, y_train, y_test)

In [ ]:
# ET CV
subs_et, fi, oof_et_predicts = run_model_cv(et_model, X=X.loc[:, features], y=y, sub_test=X_sub.loc[:, features], cv=5, random_state=RANDOM_SEED)

### LGBM

In [ ]:
# LGBM
def lgbm_model(X_train, X_test, y_train, y_test):
    # LGBM
    lgbm_model = LGBMRegressor(objective='regression',
                              num_leaves=4,
                              learning_rate=0.1, 
                              n_estimators=5000,
                              max_bin=75, 
                              bagging_fraction=0.8,
                              bagging_freq=9, 
                              feature_fraction=0.45,
                              feature_fraction_seed=9, 
                              bagging_seed=12,
                              min_data_in_leaf=3, 
                              min_sum_hessian_in_leaf=2).fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric='mape')

    return (lgbm_model)

base_lgbm_model = lgbm_model(X_train, X_test, y_train, y_test)

In [ ]:
# LGBM CV
subs_lgbm, fi, oof_lgbm_predicts = run_model_cv(lgbm_model, X=X.loc[:, :], y=y, sub_test=X_sub.loc[:, :], cv=5, random_state=RANDOM_SEED, name='lgbm', target_encoding=True, te_cols=cat_cols)

### XGBoost

In [ ]:
def xgb_model(X_train, X_test, y_train, y_test):
    xgb_reg = xgb.XGBRegressor(n_estimators=5000, learning_rate=0.1)
    xgb_reg.fit(X_train, y_train, early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], verbose=False)

    return (xgb_reg)

base_xgb_model = xgb_model(X_train, X_test, y_train, y_test)

In [ ]:
subs_xgb, fi, oof_xgb_predicts = run_model_cv(xgb_model, X=X.loc[:, :], y=y, sub_test=X_sub.loc[:, :], cv=5, random_state=RANDOM_SEED, name='xgb')

## Bagging

In [ ]:
def bagging_model(X_train, X_test, y_train, y_test):
    # Create a based model
    bagging_clf = BaggingRegressor(DecisionTreeRegressor(random_state=RANDOM_SEED), n_jobs=-1, n_estimators=200)
    bagging_clf.fit(X_train, y_train)

    return (bagging_clf)

subs_bagging, fi = run_model_cv(bagging_model, X=X.loc[:, features], y=y, sub_test=X_sub.loc[:, features], cv=5, random_state=RANDOM_SEED, name='bagging', model_type='bagging')

In [ ]:
report = show_models_report()
report

In [ ]:
# Merge Submissions
def average_submission(*args, k=0.95):
    sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

    subs = np.zeros_like(args[0])
    for submission in args:
        subs += submission / len(args)

    subs['blend'] = (subs.sum(axis=1))/len(subs.columns)*k
    sample_submission['price'] = round_preds(subs['blend'].values)
    sample_submission.to_csv(f'submission_blend_v{VERSION}.csv', index=False)
        
    return sample_submission

In [ ]:
# StackingModels
k = 0.95

X_train_oof = pd.DataFrame(np.c_[oof_et_predicts, oof_rf_predicts], columns=['et', 'rf'])
X_test_oof = pd.DataFrame(np.c_[subs_et.mean(axis=1)*k, sub_rf.mean(axis=1)*k], columns=['et', 'rf'])


# MetaModel
def meta_model(X_train, y_train):
    # Create a based model
    meta = Ridge()
    meta.fit(X_train, y_train)

    return (meta)

meta_clf = meta_model(X_train_oof, y.values)
subs = meta_clf.predict(X_test_oof)

sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')
sample_submission['price'] = subs
sample_submission.to_csv(f'submission_stack_v{VERSION}.csv', index=False)
sample_submission.head()

# Submission

In [ ]:
sample_submission = average_submission(sub_cat)
sample_submission.head(10)


## Разберем ошибки моделей

In [ ]:
# Предикты моделей
selected_columns = X.columns.str.contains('dummy_')
predicts_df = pd.DataFrame(np.c_[oof_etfs_predicts, oof_cat_predicts], columns=['price_ET', 'price_CAT'])
model_error_df = pd.concat([X[X.columns[~selected_columns]].reset_index(drop=True), y.reset_index(drop=True), predicts_df], axis=1)

# Столбцы с MAPE
model_error_df['mape_et'] = mape_per_row(model_error_df.price, model_error_df.price_ET)
model_error_df['mape_cat'] = mape_per_row(model_error_df.price, model_error_df.price_CAT)
model_error_df['mape_mean'] = (model_error_df['mape_et'] + model_error_df['mape_cat']) / 2

columns_rename = {'Владельцы': 'owners', 'Владение': 'ownage', 'ПТС': 'pts', 'Привод': 'gear_type', 'Руль': 'wheeldrive', 'Состояние': 'health', 'Таможня': 'custom'}                
orig_cats.rename(columns=columns_rename, inplace=True)

model_error_df[orig_cats.drop(['owners'], axis=1).columns] = orig_cats.drop(['owners'], axis=1).reset_index(drop=True)
model_error_df.drop_duplicates(inplace=True)

In [ ]:
# Общее количество записей с высокой ошибкой [11576, 11511]
print(f'Общее количество предсказаний с высоким MAPE: {model_error_df[model_error_df.mape_mean > 20].brand.count()}')

In [ ]:
# Посмортим на MAPE по маркам (отфильтрованы "плохие брэнды")
brand_mape = plot_feature_by_avg_target(model_error_df, 'brand', target='mape_cat', plot=True, title='MAPE (CAT)')
brand_mape = plot_feature_by_avg_target(model_error_df, 'brand', target='mape_et', plot=True, title='MAPE (ExtraTree)')

**Предварительно были отфильтрованы марки авто, на которых был очень высокий MAPE.**
1. Модели ошибаются на марке PROTON, а ET ошибается на HUANGHAI. Кандидаты на удаление из выборки.
2. На втором этапе видно, что много ошибок FERRARI, DW, DATSUN, IVECO, LIFAN, MASERATI, MCLAREN, TESLA

In [ ]:
# Посмортим на MAPE по типу топлива
plot_feature_by_avg_target(model_error_df, 'fuelType', target='mape_cat', plot=True, title='MAPE (CAT)')
plot_feature_by_avg_target(model_error_df, 'fuelType', target='mape_et', plot=True, title='MAPE (ExtraTree)')

Выше всего ошибка на электрокарах. Обе модели показывают одинаковые результаты.

In [ ]:
# TestBrand MAPE
test_brands = ['BMW',
 'VOLKSWAGEN',
 'NISSAN',
 'MERCEDES',
 'TOYOTA',
 'AUDI',
 'MITSUBISHI',
 'SKODA',
 'VOLVO',
 'HONDA',
 'INFINITI',
 'LEXUS']

# Наши тестовые марки
plot_feature_by_avg_target(model_error_df[model_error_df.brand.isin(test_brands)], 'brand', target='mape_cat', plot=True, title='MAPE (CAT)')
plot_feature_by_avg_target(model_error_df[model_error_df.brand.isin(test_brands)], 'brand', target='mape_et', plot=True, title='MAPE (ExtraTree)')



Как видим, хуже всего модели предсказывают Японские брэнды. Нужно с этим разобраться.

In [ ]:
# Посмотрим по типу корпуса по худшим маркам
# HONDA
plot_feature_by_avg_target(model_error_df[model_error_df.brand == 'TOYOTA'], 'model_name', target='mape_cat', plot=True, title='MAPE (Cat)')
plot_feature_by_avg_target(model_error_df[model_error_df.brand == 'TOYOTA'], 'bodyType', target='mape_cat', plot=True, title='MAPE (Cat)')
plot_feature_by_avg_target(model_error_df[model_error_df.brand == 'TOYOTA'], 'model_name', target='mape_et', plot=True, title='MAPE (ET)')
plot_feature_by_avg_target(model_error_df[model_error_df.brand == 'TOYOTA'], 'bodyType', target='mape_et', plot=True, title='MAPE (ET)')



In [ ]:

# TOYOTA
plot_feature_by_avg_target(model_error_df[model_error_df.brand == 'HONDA'], 'model_name', target='mape_cat', plot=True, title='MAPE (Cat)')
plot_feature_by_avg_target(model_error_df[model_error_df.brand == 'HONDA'], 'bodyType', target='mape_cat', plot=True, title='MAPE (Cat)')
plot_feature_by_avg_target(model_error_df[model_error_df.brand == 'HONDA'], 'model_name', target='mape_et', plot=True, title='MAPE (ET)')
plot_feature_by_avg_target(model_error_df[model_error_df.brand == 'HONDA'], 'bodyType', target='mape_et', plot=True, title='MAPE (ET)')

In [ ]:
# Отберем худшие предсказания моделей HONDA TOYOTA
filter_worst = np.where((model_error_df.brand.isin(['TOYOTA', 'HONDA'])) & (model_error_df.mape_mean > 20))
worst_brands = model_error_df.iloc[filter_worst]


worst_brands.head()

In [ ]:
shap_values = base_cat_model.get_feature_importance(Pool(X, label=y), 
                                                                      type="ShapValues")


In [ ]:
expected_value = shap_values[0,-1]
shap_values_ = shap_values[:,:-1]

shap.initjs()
shap.force_plot(expected_value, shap_values_[19475,:], X.iloc[19475,:])

In [ ]:
shap.summary_plot(shap_values_, X_test)


In [ ]:
target_encoder

In [ ]:
model_error_df

In [ ]:
5.547823e+05

In [ ]:
model_error_df.groupby('brand').color.count()